<a href="https://colab.research.google.com/github/bamirandab/bike-sharing/blob/develop/notebooks/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import io

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf

In [3]:
# Downloading the csv file from your GitHub account
url = "https://raw.githubusercontent.com/bamirandab/bike-sharing/develop/data/train.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
df = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [4]:
df.dropna(axis=0, inplace=True)
df

,season_1,season_2,season_3,season_4,mnth_1,mnth_10,mnth_11,mnth_12,mnth_2,mnth_3,...,weathersit_4,yr,holiday,temp,atemp,hum,windspeed,cnt,date,cnt_7
161,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0.18,0.1970,0.51,0.1642,25,2011-01-08,16.0
162,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0.18,0.2121,0.55,0.0896,16,2011-01-08,40.0
163,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0.18,0.2424,0.55,0.0000,16,2011-01-08,32.0
164,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0.18,0.1970,0.55,0.1642,7,2011-01-08,13.0
165,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0.18,0.1970,0.55,0.1642,1,2011-01-08,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0.26,0.2576,0.60,0.1642,119,2012-12-31,18.0
17375,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0.26,0.2576,0.60,0.1642,89,2012-12-31,23.0
17376,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0.26,0.2576,0.60,0.1642,90,2012-12-31,22.0
17377,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0.26,0.2727,0.56,0.1343,61,2012-12-31,12.0


In [5]:
X, y = df.drop(['cnt','date','cnt_7'],axis = 1), df.cnt

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.1, random_state=42)

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=X.shape[1]))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model.fit(X_train, y_train, epochs=100, batch_size=2**5, validation_data = (X_test, y_test))

Epoch 1/100
481/481 [==============================] - 11s 4ms/step - loss: 12867.9561 - mean_squared_error: 12867.9561 - val_loss: 2891.3469 - val_mean_squared_error: 2891.3469
Epoch 2/100
481/481 [==============================] - 2s 4ms/step - loss: 4187.3052 - mean_squared_error: 4187.3052 - val_loss: 2634.2424 - val_mean_squared_error: 2634.2424
Epoch 3/100
481/481 [==============================] - 2s 4ms/step - loss: 3587.3118 - mean_squared_error: 3587.3118 - val_loss: 3013.3435 - val_mean_squared_error: 3013.3435
Epoch 4/100
481/481 [==============================] - 2s 4ms/step - loss: 3205.5127 - mean_squared_error: 3205.5127 - val_loss: 3110.5366 - val_mean_squared_error: 3110.5366
Epoch 5/100
481/481 [==============================] - 2s 5ms/step - loss: 3058.1375 - mean_squared_error: 3058.1375 - val_loss: 2102.9790 - val_mean_squared_error: 2102.9790
Epoch 6/100
481/481 [==============================] - 2s 5ms/step - loss: 3002.2083 - mean_squared_error: 3002.2083 - val